<a href="https://colab.research.google.com/github/VertaAI/modeldb-client/blob/development/workflows/demos/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fully-Connected Network (TensorFlow)

In [ ]:
#Run this cell if you are running this python notebook on Google Colab and restart your notebook when prompted
!pip install verta

In [ ]:
import os, sys

import numpy as np

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

from verta import ModelDBClient


data_dir = os.path.join("..", "data", "mnist")
output_dir = os.path.join("..", "output", "tensorflow")
os.makedirs(data_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [ ]:
HOST = 
PORT = 

---

# Log Workflow

## Instantiate Client

In [ ]:
client = ModelDBClient(HOST, PORT)
proj = client.set_project("MNIST Multiclassification")
expt = client.set_experiment("FC-NN")

## Begin Experiment Run

In [ ]:
run = client.set_experiment_run("Test Run")

# define everything we need to run this experiment
TRAIN_DATA_PATH = os.path.join(data_dir, "train.npz")
TEST_DATA_PATH = os.path.join(data_dir, "test.npz")
VAL_PLOT_PATH = os.path.join(output_dir, "val_obs.png")
MODEL_PATH = os.path.join(output_dir, "tensorflow-basic.hdf5")

HIDDEN_SIZE = 512
DROPOUT = 0.2

OPTIMIZER = 'adam'
LOSS = 'sparse_categorical_crossentropy'

BATCH_SIZE = 1024
NUM_EPOCHS = 2
VALIDATION_SPLIT = 0.1

## Define and Log Model

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(HIDDEN_SIZE, activation=tf.nn.relu))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

run.log_hyperparameter("hidden_size", HIDDEN_SIZE)
run.log_hyperparameter("dropout", DROPOUT)

## Compile and Log Training Procedure

In [ ]:
model.compile(optimizer=OPTIMIZER,
              loss=LOSS,
              metrics=['accuracy'])

run.log_hyperparameter("optimizer", OPTIMIZER)
run.log_hyperparameter("loss", LOSS)

## Load and Log Training Data

In [ ]:
train_data = np.load(TRAIN_DATA_PATH)
run.log_dataset("train_data", TRAIN_DATA_PATH)
run.log_dataset("test_data", TEST_DATA_PATH)

X_train, y_train = train_data['X'], train_data['y']

## Run and Log Validation

In [ ]:
run.log_hyperparameter("batch_size", BATCH_SIZE)
run.log_hyperparameter("num_epochs", NUM_EPOCHS)
run.log_hyperparameter("validation_split", VALIDATION_SPLIT)

def log_validation(epoch, logs):  # Keras will call this each epoch
    run.log_observation("val_train_loss", float(logs['loss']))
    run.log_observation("val_train_acc", float(logs['acc']))
    run.log_observation("val_loss", float(logs['val_loss']))
    run.log_observation("val_acc", float(logs['val_acc']))

_ = model.fit(X_train, y_train, validation_split=VALIDATION_SPLIT,
              batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
              callbacks=[keras.callbacks.LambdaCallback(on_epoch_end=log_validation)])

## Produce and Log Accuracy Plot

In [ ]:
# retrieve observations from our ExperimentRun instance
plt.plot(run.get_observations("val_acc"), label="val")
plt.plot(run.get_observations("val_train_acc"), label="train")

plt.ylim(0, 1)
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend(loc='best')

plt.savefig(VAL_PLOT_PATH)
run.log_image("validation_plot", VAL_PLOT_PATH)

plt.show()

## Save and Log Model

In [ ]:
keras.models.save_model(model, MODEL_PATH)
run.log_model("model", MODEL_PATH)

---

# Load Workflow

## Instantiate Client

In [ ]:
client = ModelDBClient(HOST, PORT)
proj = client.set_project("MNIST Multiclassification")
expt = client.set_experiment("FC-NN")

## Load Experiment Run

In [ ]:
run = client.set_experiment_run("Test Run")

# retrieve everything we need from our ExperimentRun instance
datasets = run.get_datasets()
models = run.get_models()
hyperparameters = run.get_hyperparameters()

TRAIN_DATA_PATH = datasets['train_data']
TEST_DATA_PATH = datasets['test_data']
VAL_PLOT_PATH = run.get_image("validation_plot")
MODEL_PATH = models['model']

HIDDEN_SIZE = hyperparameters['hidden_size']
DROPOUT = hyperparameters['dropout']

OPTIMIZER = hyperparameters['optimizer']
LOSS = hyperparameters['loss']

BATCH_SIZE = hyperparameters['batch_size']
NUM_EPOCHS = hyperparameters['num_epochs']
VALIDATION_SPLIT = hyperparameters['validation_split']

## Restore Accuracy Plot

In [ ]:
img = plt.imread(VAL_PLOT_PATH)
plt.figure(figsize=(8, 6))
plt.axis('off')

img_plt = plt.imshow(img)

## Restore Model Definition

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(HIDDEN_SIZE, activation=tf.nn.relu))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

## Restore Training Procedure Definition

In [ ]:
model.compile(optimizer=OPTIMIZER,
              loss=LOSS,
              metrics=['accuracy'])

## Restore Training Data

In [ ]:
train_data = np.load(TRAIN_DATA_PATH)
X_train, y_train = train_data['X'], train_data['y']

## Retrain Model on Full Training Set

In [ ]:
def log_test(epoch, logs):  # Keras will call this each epoch
    run.log_observation("train_loss", float(logs['loss']))
    run.log_observation("train_acc", float(logs['acc']))

_ = model.fit(X_train, y_train,
              batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
              callbacks=[keras.callbacks.LambdaCallback(on_epoch_end=log_test)])

## Run and Log Testing

In [ ]:
test_data = np.load(TEST_DATA_PATH)
X_test, y_test = test_data['X'], test_data['y']

test_loss, test_acc = model.evaluate(X_test, y_test)
print("test_loss: {:.4f} - test_acc: {:.4f}".format(test_loss, test_acc))

run.log_metric("test_loss", test_loss)
run.log_metric("test_acc", test_acc)